In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# Read data

In [2]:
df = pd.read_csv('data/sales_train_v2.csv')
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


Print dtypes and memory consumption

Downcast dtypes to save memory

In [3]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

df = downcast_dtypes(df)
#df.info()

# Aggregations

Sum sales for each (item, shop) pair 

In [ ]:
'''grid = []
for date_block in df['date_block_num'].unique():
    
    # cartesian product of shops x items in each month
    cur_shops = df[df['date_block_num'] == date_block]['shop_id'].unique()
    cur_items = df[df['date_block_num'] == date_block]['item_id'].unique()
    grid.append(np.array(list(itertools.product(*[cur_shops, cur_items, [date_block]])),dtype='int32'))'''

In [ ]:
#grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [5]:
df['item_cnt_day'] = df['item_cnt_day'].clip(0,20)
gb_cnt = df.groupby(index_cols)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})
gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)

In [6]:
gb_cnt.head()

,shop_id,item_id,date_block_num,item_cnt_month
0,0,30,1,20
1,0,31,1,11
2,0,32,0,6
3,0,32,1,10
4,0,33,0,3


Join aggregated data to the grid

In [7]:
df = pd.merge(df, gb_cnt, how='left', on=index_cols).fillna(0)
df['item_cnt_month'] = df['item_cnt_month'].astype(int)
df = downcast_dtypes(df)

# Combine all data sources

In [8]:
items = pd.read_csv('data/items.csv')
shops = pd.read_csv('data/shops.csv')
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


Join items with train set

In [12]:
df = pd.merge(df, items[['item_id', 'item_category_id']], on = 'item_id')
df[df['item_id']==13315].head(20)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,item_category_id_x,item_category_id_y,item_category_id
2916235,21.08.2015,31,42,13315,649.0,1.0,1,47,47,47
2916236,27.08.2015,31,25,13315,649.0,1.0,1,47,47,47
2916237,15.09.2015,32,42,13315,649.0,1.0,1,47,47,47
2916238,23.10.2015,33,45,13315,649.0,1.0,1,47,47,47
2916239,24.10.2015,33,56,13315,649.0,1.0,1,47,47,47
2916240,25.10.2015,33,57,13315,649.0,1.0,1,47,47,47
2916241,18.10.2015,33,47,13315,649.0,1.0,1,47,47,47
2916242,04.10.2015,33,20,13315,584.0,2.0,2,47,47,47


Join items with test set

In [10]:
df_test = pd.read_csv('data/test.csv', delimiter="," ,skiprows=1, usecols = (1,2), dtype=int)
df_test = pd.DataFrame(df_test, columns = ['shop_id', 'item_id'])

df_test = df_test.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')
df_test['date_block_num'] = 34
df_test['item_cnt_month'] = 0

Concatenate train and test sets, fix category names

In [ ]:
df_train_test = pd.concat([df, df_test], axis = 0) 
item_cat = pd.read_csv('data/item_categories.csv')

# Fix category
categories_list = list(item_cat.item_category_name)
for ind in range(0,1):
    categories_list[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    categories_list[ind] = 'Access'
categories_list[8] = 'Tickets (figure)'
categories_list[9] = 'Delivery of goods'
for ind in range(10,18):
    categories_list[ind] = 'Consoles'
for ind in range(18,25):
    categories_list[ind] = 'Consoles Games'
categories_list[25] = 'Accessories for games'
for ind in range(26,28):
    categories_list[ind] = 'phone games'
for ind in range(28,32):
    categories_list[ind] = 'CD games'
for ind in range(32,37):
    categories_list[ind] = 'Card'
for ind in range(37,43):
    categories_list[ind] = 'Movie'
for ind in range(43,55):
    categories_list[ind] = 'Books'
for ind in range(55,61):
    categories_list[ind] = 'Music'
for ind in range(61,73):
    categories_list[ind] = 'Gifts'
for ind in range(73,79):
    categories_list[ind] = 'Soft'
for ind in range(79,81):
    categories_list[ind] = 'Office'
for ind in range(81,83):
    categories_list[ind] = 'Clean'
categories_list[83] = 'Elements of a food'

label_encoder = LabelEncoder()
#item_cat['item_category_name_int'] = label_encoder.fit_transform(item_cat['item_category_name'])

item_cat['item_category_id_fix'] = label_encoder.fit_transform(categories_list)
item_cat['item_category_name_fix'] = categories_list
df_train_test = df_train_test.merge(item_cat[['item_category_id', 'item_category_id_fix']], on = 'item_category_id', how = 'left')
_ = df_train_test.drop(['item_category_id'],axis=1, inplace=True)
df_train_test.rename(columns = {'item_category_id_fix':'item_category_id'}, inplace = True)

_ = item_cat.drop(['item_category_id'],axis=1, inplace=True)
_ = item_cat.drop(['item_category_name'],axis=1, inplace=True)

item_cat.rename(columns = {'item_category_id_fix':'item_category_id'}, inplace = True)
item_cat.rename(columns = {'item_category_name_fix':'item_category_name'}, inplace = True)
item_cat = item_cat.drop_duplicates()
item_cat.index = np.arange(0, len(item_cat))


item_cat.head()

In [ ]:
df_train_test = df_train_test.merge(item_cat, on = 'item_category_id', how = 'left')
df_train_test['item_category_name'].fillna(value='Other', inplace=True)
df_train_test['item_category_id'].fillna(value=-1, inplace=True)
df_train_test.head()

In [ ]:
df_train_test['item_cnt_month'] = df_train_test['item_cnt_month'].astype(np.int16)
df_train_test['item_id'].fillna(value=-1, inplace=True)
df_train_test['item_id'] = df_train_test['item_id'].astype(np.int32)
df_train_test['shop_id'].fillna(value=-1, inplace=True)
df_train_test['shop_id'] = df_train_test['shop_id'].astype(np.int32)
df_train_test['item_category_id'] = df_train_test['item_category_id'].astype(np.int16)
df = downcast_dtypes(df_train_test)
df.head()

In [ ]:
df.to_feather('df_preprocessed')

In [ ]:
df = pd.read_feather('df_preprocessed')

Don't use the oldest data. Split the data into train, validation and test set

In [ ]:
df = df[df['date_block_num'] >= 12]
df_test = df[df['date_block_num'] == 34]
df_val = df[df['date_block_num'] == 33]
df_train = df[df['date_block_num'] <= 32]

Lag features: number of items sold in previous month for shop, item and category

In [ ]:
lag_columns = ['shop_id', 'item_id', 'item_category_id']
lag = 1

for lag_column in lag_columns:

    df_lag = df.groupby([lag_column, 'date_block_num'])['item_cnt_month'].sum().reset_index()
    
    print(f'Grouped by: {lag_column}')
    print(df_lag.head())

    df['merge_month'] = df['date_block_num'] - lag
    df = pd.merge(df, df_lag, how='left', left_on=[lag_column, 'merge_month'], right_on=[lag_column, 'date_block_num']).fillna(0)
    df.rename(columns={'item_cnt_month_y': lag_column + '_lag_' + str(lag)}, inplace=True)
    df.rename(columns={'item_cnt_month_x': 'item_cnt_month'}, inplace=True)
    df.rename(columns={'date_block_num_x': 'date_block_num'}, inplace=True)
    df.drop(columns='date_block_num_y', inplace=True)
    df.drop(columns='merge_month', inplace=True)


df.head()

Mean item price from previous month

In [ ]:
lag = 1

df_lag = df.groupby(['item_id', 'date_block_num'])['item_price'].mean()
df_lag.head()
df['merge_month'] = df['date_block_num'] - lag
df = pd.merge(df, df_lag, how='left', left_on=['item_id', 'merge_month'], right_on=['item_id', 'date_block_num']).fillna(0)
df.rename(columns={'item_price_y': 'previous_month_price'}, inplace=True)
df.rename(columns={'item_price_x': 'item_price'}, inplace=True)
df.drop(columns='merge_month', inplace=True)

In [ ]:
df.head()

Lag feature: monthly count for (item, shop) pairs

In [ ]:
lag = 12

df_lag = df.groupby(['item_id', 'shop_id', 'date_block_num'])['item_cnt_month'].sum()
df['merge_month'] = df['date_block_num'] - lag
df = pd.merge(df, df_lag, how='left', left_on=['item_id', 'shop_id', 'merge_month'], right_on=['item_id', 'shop_id', 'date_block_num']).fillna(0)
df.rename(columns={'item_cnt_month_y': 'item_shop_lag_' + str(lag)}, inplace=True)
df.rename(columns={'item_cnt_month_x': 'item_cnt_month'}, inplace=True)
df.drop(columns='merge_month', inplace=True)

In [ ]:
df.to_csv('df_preprocessed.csv')

In [ ]:
small_idx = np.random.permutation(range(1882486))[:40000]
small_df = df.iloc[small_idx]
small_df = small_df.reset_index()
valid_idx = small_df[small_df['date_block_num'] == 33].index.values

In [ ]:
from fastai.tabular import *

#df = downcast_dtypes(df)


procs=[Categorify, Normalize]

cat_vars = ['shop_id', 'item_id', 'item_category_id']
cont_vars = ['item_price', 'shop_id_lag_1',
            'item_id_lag_1', 'item_category_id_lag_1', 'shop_id_lag_2',
            'item_id_lag_2', 'item_category_id_lag_2', 'shop_id_lag_3',
            'item_id_lag_3', 'item_category_id_lag_3', 'shop_id_lag_12',
            'item_id_lag_12', 'item_category_id_lag_12', 'previous_month_price',
            'item_shop_lag_1', 'item_shop_lag_2', 'item_shop_lag_3', 'date_block_num']

dep_var = 'item_cnt_month'
small_df = small_df[cat_vars + cont_vars + [dep_var]]

In [ ]:
path = 'data'

In [ ]:
data = (TabularList.from_df(small_df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList)
                .databunch())

In [ ]:
emb_szs= {
            'shop_id': 15,
            'item_id': 15,
            'item_category_id': 10,
        }

In [ ]:
y_range = torch.tensor([0, 20.5], device=defaults.device)

In [ ]:
learn = tabular_learner(data, layers=[800,400], ps=[0.001,0.01], emb_drop=0.04, y_range=y_range, metrics=rmse)

In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 5e-3, wd=0.2)

In [ ]:
learn.save('1')

In [ ]:
learn.recorder.plot_losses(skip_start=10)

In [ ]:
preds = learn.get_preds()[0].numpy()